# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/Users/rene/work/practice/udacity/data_engineering_gym/streaming_app_etc_cassandra


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
print(full_data_rows_list[1])

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056
['The Prodigy', 'Logged In', 'Ryan', 'M', '1', 'Smith', '260.07465', 'free', 'San Jose-Sunnyvale-Santa Clara, CA', 'PUT', 'NextSong', '1.54102E+12', '583', 'The Big Gundown', '200', '1.54224E+12', '26']


In [4]:
# check the number of rows in your csv file, now some rows were empty!
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
import logging
from cassandra.cluster import Cluster

logging.basicConfig(level=logging.INFO)

# Make connection
def cassandra_connection():
    """
    Connection object for Cassandra
    :return: session, cluster
    """
    cluster = Cluster(['127.0.0.1'], port=9042)
    session = cluster.connect()
    return session, cluster

session, cluster  = cassandra_connection()


INFO:cassandra.policies:Using datacenter 'datacenter' for DCAwareRoundRobinPolicy (via host '127.0.0.1:9042'); if incorrect, please specify a local_dc to the constructor, or limit contact points to local cluster nodes


#### Create Keyspace

In [7]:
# TO-DO: Create a Keyspace 

try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [8]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




### Query 1
#### Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession = 4

In [9]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

query1 = "CREATE TABLE IF NOT EXISTS session_item "
query1 = query1 + "(sessionId text, itemInSession text, artist text, song text, length float,\
                    PRIMARY KEY (sessionId, itemInSession));"
try:
    session.execute(query1)
except Exception as e:
    print(e)                    

In [10]:
columns_names_ordered = ['artist','firstName','gender','itemInSession',
'lastName','length', 'level','location','sessionId','song','userId']

In [11]:
# Some preparations to avoid errors
columns_included_1 = ('sessionId', 'itemInSession', 'artist', 'song', 'length')

def filter_columns(columns_included):
    """
    columns_included: tuple
    """

    ids_included = tuple(columns_names_ordered.index(i) for i in columns_included)
    str_included = str(columns_included).replace("'","")

    
    return ids_included, str_included

ids_included, str_included = filter_columns(columns_included_1)

print("columns indices to include in queue: ", ids_included)
print("columns indices to include in queue: ", str_included)

columns indices to include in queue:  (8, 3, 0, 9, 5)
columns indices to include in queue:  (sessionId, itemInSession, artist, song, length)


In [12]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO session_item " + str_included
        query = query + " VALUES " + (
        str(tuple('%s' for _ in ids_included)).replace("'","") )
        data_entry = list(line[i] for i in ids_included)
        data_entry[-1] = float(data_entry[-1])
        data_entry = tuple(data_entry)
        session.execute(query, data_entry)

#### Do a SELECT to verify that the data have been inserted into each table

In [13]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table

select1 = """SELECT artist, song, length FROM session_item 
WHERE sessionId = '338' and itemInSession = '4';"""

try:
    rows = session.execute(select1)
except Exception as e:
    print(e)
    
for row in rows:
    print (row)

Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=495.30731201171875)


### Query 2

#### Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [14]:
query2 = "CREATE TABLE IF NOT EXISTS user_session"
query2 = query2 + "(userId text, sessionId text, itemInSession text, artist text, song text, firstName text, lastName text,\
                    PRIMARY KEY ((userId, sessionId), itemInSession))"
try:
    session.execute(query2)
except Exception as e:
    print(e)

In [15]:
# Some preparations to avoid errors
columns_included_2 = ('userId', 'sessionId', 'itemInSession', 'artist', 
                      'song', 'firstName', 'lastName')

ids_included, str_included = filter_columns(columns_included_2)

print("columns indices to include in queue: ", ids_included)
print("columns indices to include in queue: ", str_included)

columns indices to include in queue:  (10, 8, 3, 0, 9, 1, 4)
columns indices to include in queue:  (userId, sessionId, itemInSession, artist, song, firstName, lastName)


In [16]:

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_session " + str_included
        query = query + " VALUES " + (
        str(tuple('%s' for _ in ids_included)).replace("'", ""))
        data_entry = tuple(line[i] for i in ids_included)
        session.execute(query, data_entry)

In [17]:
select2 = """SELECT artist, song, itemInSession, song, firstName, lastName
FROM user_session 
WHERE userId = '10' and sessionId = '182';"""

try:
    rows = session.execute(select2)
except Exception as e:
    print(e)

for row in rows:
    print(row)
    print()

Row(artist='Down To The Bone', song="Keep On Keepin' On", iteminsession='0', song_="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz')

Row(artist='Three Drives', song='Greece 2000', iteminsession='1', song_='Greece 2000', firstname='Sylvie', lastname='Cruz')

Row(artist='Sebastien Tellier', song='Kilometer', iteminsession='2', song_='Kilometer', firstname='Sylvie', lastname='Cruz')

Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', iteminsession='3', song_='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')



### Query 3: 
#### Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [18]:
query3 = "CREATE TABLE IF NOT EXISTS songs_userId"
query3 = query3 + "(song text, userId text, firstName text, lastName text,\
                    PRIMARY KEY (song, userId));"
try:
    session.execute(query3)
except Exception as e:
    print(e)     

In [19]:
# Some preparations to avoid errors
columns_included_2 = ('song', 'userId', 'firstName', 'lastName')

ids_included, str_included = filter_columns(columns_included_2)

print("columns indices to include in queue: ", ids_included)
print("columns indices to include in queue: ", str_included)

columns indices to include in queue:  (9, 10, 1, 4)
columns indices to include in queue:  (song, userId, firstName, lastName)


In [20]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songs_userId " + str_included
        query = query + " VALUES " + (
        str(tuple('%s' for _ in ids_included)).replace("'", ""))
        data_entry = tuple(line[i] for i in ids_included)
        session.execute(query, data_entry)

In [21]:
select3 = """SELECT song, firstName, lastName FROM songs_userId WHERE song='All Hands Against His Own';"""

try:
    rows = session.execute(select3)
except Exception as e:
    print(e)

for row in rows:
    print(row, '\n')

Row(song='All Hands Against His Own', firstname='Jacqueline', lastname='Lynch') 

Row(song='All Hands Against His Own', firstname='Tegan', lastname='Levine') 

Row(song='All Hands Against His Own', firstname='Sara', lastname='Johnson') 



### Drop the tables before closing out the sessions

In [22]:
## TO-DO: Drop the table before closing out the sessions

query = "DROP TABLE session_item;"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE user_session;"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE songs_userId;"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [23]:
session.shutdown()
cluster.shutdown()